In [1]:
# Import libraries
import os, datetime
from glob import glob

import earthpy as et
import numpy as np
import pandas as pd


In [2]:
# Define filepath to earth-analytics/data/natureserve directory
ns_data_path = os.path.normpath(os.path.join(et.io.HOME, 
                                             'earth-analytics', 
                                             'data', 
                                             'natureserve'))

# Check if natureserve directory exists (create if needed), then change 
# working directory to 'earth-analytics/data/natureserve'
if os.path.exists(ns_data_path):
 print('Working directory is set to earth-analytics/data/natureserve.')
else:
 print("Creating new directory; working directory set to newly created "
       "'earth-analytics/data/natureserve'.")
 os.makedirs(ns_data_path)
 
os.chdir(ns_data_path)


Working directory is set to earth-analytics/data/natureserve.


In [3]:
# Import provided csv with info about selected ecosystems
readme_info_path = os.path.normpath(os.path.join(ns_data_path, 
                                                 'LF_EVT_2020_README.csv'))

readme_info = pd.read_csv(readme_info_path).set_index("EVT_Name_1")

readme_info

,OID,Value,Count_30m,US_hectare,US_km2,LFRDB,elcode,element_gl,NatureServ,NSX_link,...,A3_FINAL,B1_FINAL,B2_FINAL,C3_FINAL,D3_FINAL,RLE_FINAL,GRANK_EQUI,RED,GREEN,BLUE
EVT_Name_1,,,,,,,,,,,,,,,,,,,,,
Columbia Basin Palouse Prairie,132,7142,106116,9550,96,7142,CES304.792,722880,Columbia Basin Palouse Prairie,https://explorer.natureserve.org/Taxon/ELEMENT...,...,CR,LC,LC,EN,VU,CR,G1,204,252,105
Western Great Plains Foothill and Piedmont Grassland,137,7147,4546277,409165,4092,7147,CES303.817,722856,Western Great Plains Foothill and Piedmont Gra...,https://explorer.natureserve.org/Taxon/ELEMENT...,...,VU,LC,LC,NT,LC,VU,G3,218,238,243
Crowley's Ridge Mesic Loess Slope Forest,224,7322,1090956,98186,982,7322,CES203.079,798100,Crowley's Ridge Mesic Loess Slope Forest,https://explorer.natureserve.org/Taxon/ELEMENT...,...,LC,EN,EN,VU,CR,CR (EN-CR),G1G2,144,201,143
East Gulf Coastal Plain Northern Loess Bluff Forest,229,7327,2050154,184514,1845,7327,CES203.481,723105,East Gulf Coastal Plain Northern Loess Bluff F...,https://explorer.natureserve.org/Taxon/ELEMENT...,...,EN,LC,LC,VU-EN,EN,EN,G2,149,143,26
Atlantic Coastal Plain Fall-line Sandhills Longleaf Pine Woodland,244,7346,5015841,451426,4514,7346,CES203.254,723231,Atlantic Coastal Plain Fall-line Sandhills Lon...,https://explorer.natureserve.org/Taxon/ELEMENT...,...,EN,LC,LC,VU,CR,EN (EN-CR),G1G2,70,96,32
Central Tallgrass Prairie,314,7421,10225903,920331,9203,7421,CES205.683,722976,Central Tallgrass Prairie,https://explorer.natureserve.org/Taxon/ELEMENT...,...,CR,LC,LC,DD,CR,CR,G1,243,201,28
Southwest Florida Dune and Coastal Grassland,323,7431,19717,1775,18,7431,CES203.539,723063,Southwest Florida Dune and Coastal Grassland,https://explorer.natureserve.org/Taxon/ELEMENT...,...,DD,LC,LC,NE,CR,CR,G1,245,252,179
South Florida Cypress Dome,335,7447,900234,81021,810,7447,CES411.365,723151,South Florida Cypress Dome,https://explorer.natureserve.org/Taxon/ELEMENT...,...,DD,VU,LC,DD,LC,VU,G3,54,163,120
Northern Atlantic Coastal Plain Tidal Salt Marsh,676,9197,1634510,147106,1471,9197,CES203.519,723073,Northern Atlantic Coastal Plain Tidal Salt Marsh,https://explorer.natureserve.org/Taxon/ELEMENT...,...,EN,LC,LC,DD,EN,EN,G2,131,173,223


In [4]:
# Add column 'Type' to 'readme_info' dataframe (needed for Marxan input file 
# 'targets.csv'). Uses np.select to assign a number based upon the 'RLE_FINAL' 
# string value (1 if 'CR', 'CR (CR-EN)', 'EN (CR-EN) or 'EN', 2 if 'VU')

# create a list of conditions
type_conditions = [(readme_info['RLE_FINAL'] == 'CR'), 
                  (readme_info['RLE_FINAL'] == 'CR (EN-CR)'),
                  (readme_info['RLE_FINAL'] == 'EN'),
                  (readme_info['RLE_FINAL'] == 'EN (EN-CR)'),
                  (readme_info['RLE_FINAL'] == 'VU')]

# create a list of the values to assign for each condition
type_values = [1, 1, 1, 1, 2]

# create new column using np.select to assign values using lists as arguments
readme_info['Type'] = np.select(type_conditions, type_values)

# Add column 'IUCN_Threshold' to 'readme_info' dataframe, using np.select to 
# assign a % based upon the column 'Type' (5% if 1, 10% if 2)

# create a list of conditions
current_threshold_conditions = [(readme_info['Type'] == 1), 
                               (readme_info['Type'] == 2)]

# create a list of the values to assign for each condition
current_threshold_values = [.05, .10]

# create new column using np.select to assign values using lists as arguments
readme_info['Current_IUCN_TH'] = np.select(
    current_threshold_conditions, current_threshold_values)

readme_info

,OID,Value,Count_30m,US_hectare,US_km2,LFRDB,elcode,element_gl,NatureServ,NSX_link,...,B2_FINAL,C3_FINAL,D3_FINAL,RLE_FINAL,GRANK_EQUI,RED,GREEN,BLUE,Type,Current_IUCN_TH
EVT_Name_1,,,,,,,,,,,,,,,,,,,,,
Columbia Basin Palouse Prairie,132,7142,106116,9550,96,7142,CES304.792,722880,Columbia Basin Palouse Prairie,https://explorer.natureserve.org/Taxon/ELEMENT...,...,LC,EN,VU,CR,G1,204,252,105,1,0.05
Western Great Plains Foothill and Piedmont Grassland,137,7147,4546277,409165,4092,7147,CES303.817,722856,Western Great Plains Foothill and Piedmont Gra...,https://explorer.natureserve.org/Taxon/ELEMENT...,...,LC,NT,LC,VU,G3,218,238,243,2,0.10
Crowley's Ridge Mesic Loess Slope Forest,224,7322,1090956,98186,982,7322,CES203.079,798100,Crowley's Ridge Mesic Loess Slope Forest,https://explorer.natureserve.org/Taxon/ELEMENT...,...,EN,VU,CR,CR (EN-CR),G1G2,144,201,143,1,0.05
East Gulf Coastal Plain Northern Loess Bluff Forest,229,7327,2050154,184514,1845,7327,CES203.481,723105,East Gulf Coastal Plain Northern Loess Bluff F...,https://explorer.natureserve.org/Taxon/ELEMENT...,...,LC,VU-EN,EN,EN,G2,149,143,26,1,0.05
Atlantic Coastal Plain Fall-line Sandhills Longleaf Pine Woodland,244,7346,5015841,451426,4514,7346,CES203.254,723231,Atlantic Coastal Plain Fall-line Sandhills Lon...,https://explorer.natureserve.org/Taxon/ELEMENT...,...,LC,VU,CR,EN (EN-CR),G1G2,70,96,32,1,0.05
Central Tallgrass Prairie,314,7421,10225903,920331,9203,7421,CES205.683,722976,Central Tallgrass Prairie,https://explorer.natureserve.org/Taxon/ELEMENT...,...,LC,DD,CR,CR,G1,243,201,28,1,0.05
Southwest Florida Dune and Coastal Grassland,323,7431,19717,1775,18,7431,CES203.539,723063,Southwest Florida Dune and Coastal Grassland,https://explorer.natureserve.org/Taxon/ELEMENT...,...,LC,NE,CR,CR,G1,245,252,179,1,0.05
South Florida Cypress Dome,335,7447,900234,81021,810,7447,CES411.365,723151,South Florida Cypress Dome,https://explorer.natureserve.org/Taxon/ELEMENT...,...,LC,DD,LC,VU,G3,54,163,120,2,0.10
Northern Atlantic Coastal Plain Tidal Salt Marsh,676,9197,1634510,147106,1471,9197,CES203.519,723073,Northern Atlantic Coastal Plain Tidal Salt Marsh,https://explorer.natureserve.org/Taxon/ELEMENT...,...,LC,DD,EN,EN,G2,131,173,223,1,0.05


In [5]:
# Create lists to use for iteration loops

# Create list of threshold values to test
test_threshold = [1.0, 0.75, 0.50, 0.25]

# Create alphabetical list of ecosystems to be analyzed, taken from the 
# 'EVT_Name_1' field of eco_info.csv
eco_list = readme_info.index.values.tolist()
eco_list.sort()

print(eco_list)

['Atlantic Coastal Plain Fall-line Sandhills Longleaf Pine Woodland', 'Central Tallgrass Prairie', 'Columbia Basin Palouse Prairie', "Crowley's Ridge Mesic Loess Slope Forest", 'East Gulf Coastal Plain Northern Loess Bluff Forest', 'Northern Atlantic Coastal Plain Tidal Salt Marsh', 'South Florida Cypress Dome', 'Southwest Florida Dune and Coastal Grassland', 'Western Great Plains Foothill and Piedmont Grassland']


In [6]:
# create function to write targets.csv files, for each threshold test value

def create_targets_files(eco, thresholds_test, eco_info, path):
 # YOUR CODE HERE
     """creates the targets.csv files needed for Marxan analysis.

     Parameters
     ----------
     eco : str
     name of ecosystem that will be analyzed by Marxan

     thresholds_test : list
     list of threshold values to be tested for each ecosystem

     eco_info : dataframe
     source of info for each ecosystem, with columns 'OID' (Unique ID number),
     'Name' (ecosystem name), Type (number representing RLE Status), Size of 
     Ecosystem (units of area measurement) and the Current IUCN Threshold 
     value, based upon ecosystem's RLE status

     path : filepath
     filepath to ecosystem subdirectory where targets files will be saved

     Returns
     -------
     returned_data : csv
     csv files are saved to ecosystem directories, one file for each threshold
     value to be tested
     """
     for val in thresholds_test:
            target_info = {'Id': [eco_info.loc[eco]['OID']], 
                           'Name': [eco], 
                           'Type': [eco_info.loc[eco]['Type']], 
                           'sq_km': [eco_info.loc[eco]['US_km2']],
                           'iucn_th': [eco_info.loc[eco]['Current_IUCN_TH']]}
            target_df = pd.DataFrame(data=target_info).set_index('Id')
            target_df['Target'] = (target_df['sq_km'] * target_df['iucn_th'])
            target_df['Target'] = (val * target_df['Target'])
            target_df.drop(["sq_km", "iucn_th"], axis = 1, inplace = True)
            outpath = os.path.join(path, 'targets_' +str(val) + '.csv')
            target_df.to_csv(outpath)
     return 
   

In [7]:
# Create a timestamped directory, with a subdirectory for each ecosystem 
# listed in input .csv file.  Each ecosystem directory includes the input and 
# output directories and targets.csv files for each test threshold value
# needed for Marxan analysis.

newdir = os.path.normpath(os.path.join(ns_data_path, 
                                       datetime.datetime.now().strftime(
                                           '%Y%m%d_%H%M%S')))
os.makedirs(newdir)
os.chdir(newdir)
for eco in eco_list:
    os.makedirs(eco),
    eco_data_path = os.path.normpath(os.path.join(ns_data_path, newdir, eco))
    os.chdir(eco_data_path)
    os.makedirs('input')
    os.makedirs('output')
    create_targets_files(eco, test_threshold, readme_info, eco_data_path)
    os.chdir(newdir)
    
os.getcwd()

'C:\\Users\\cwsnd\\earth-analytics\\data\\natureserve\\20220424_102615'